# Posterior samples

To get posterior samples we need to do Fisher analysis of our events and then apply priors in post-processing

In [1]:
from tools import *
import numpy as np
import pickle

In [2]:
PATH_TO_LVK_DATA = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/GWTC_LVK_data/'
PATH_TO_INFO = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/info/'
PATH_TO_RESULTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/results/'
PATH_TO_INJECTIONS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/injections/'
PATH_TO_YAML = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/yamls/'
PATH_TO_PSD = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_results/gwtc_psds/'

We are working with BBH data and we take the median as the value to inject in `gwfish`

In [3]:
waveform = 'IMRPhenomXPHM'
estimator = 'median'
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

We consider the events' list for which we have the analytic prior from the LVK analysis

In [4]:
events = np.loadtxt(PATH_TO_INFO + 'events_wf_%s_priors_%s.txt' %(estimator, waveform), dtype=str)
with open(PATH_TO_INFO + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

Specify the folder where to save results

In [5]:
results_folder = 'gwfish_medians'

The fisher parameters are the same as the injection parameters

In [ ]:
fisher_parameters = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']
gwfish_analysis(PATH_TO_INJECTIONS, PATH_TO_YAML, PATH_TO_RESULTS + results_folder + '/', 
                events, waveform, estimator, detectors, fisher_parameters)

We want to produce 3 different sets of posterior samples:
1. LVK posterior samples as obtained using full Bayesian analysis
2. Fisher matrix posterior samples, sampled from the multi-variate Gaussian likelihood approximation
3. Prior-informed Fisher matrix posterior samples, obtained sampling from the truncated multi-variate Gaussian likelihood where each sample is weighted by its prior probability

### LVK samples

In [18]:
samples_len = {}
for event in events:
    samples_lvk = get_lvk_samples(PATH_TO_LVK_DATA, event, params)
    samples_lvk_df = pd.DataFrame(samples_lvk)
    samples_len[event] = len(samples_lvk_df)
    samples_lvk_df.to_hdf(PATH_TO_RESULTS + 'posterior_samples/lvk_samples/lvk_samples_%s.hdf5' %event, mode='w', key='root')

In [20]:
with open(PATH_TO_INFO + 'lvk_samples_len_%s.pkl' %waveform, 'wb') as f:
        pickle.dump(samples_len, f)

### GWFish samples

In [21]:
with open(PATH_TO_INFO + 'lvk_samples_len_%s.pkl' %waveform, 'rb') as f:
    samples_len = pickle.load(f)

In [22]:
with open(PATH_TO_INFO + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [23]:
lbs_errs = ['snr', 'chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn',
        'psi', 'phase', 'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl',
        'err_chirp_mass', 'err_mass_ratio', 'err_luminosity_distance', 'err_dec', 'err_ra',
        'err_theta_jn', 'err_psi', 'err_phase', 'err_geocent_time', 'err_a_1', 'err_a_2', 'err_tilt_1',
        'err_tilt_2', 'err_phi_12', 'err_phi_jl', 'err_sky_location']

In [ ]:
np.random.seed(42) # for reproducibility

for event in events:
    label_err = get_label(detectors, event, estimator, 0, 'errors')
    label_cov = get_label(detectors, event, estimator, 0, 'inv_fishers')
    data = pd.read_csv(PATH_TO_RESULTS + 'gwfish_medians/' + event + '/' + label_err, names = lbs_errs, delimiter=' ', skiprows=1)
    mns = data[params].iloc[0].to_numpy()
    cov = np.load(PATH_TO_RESULTS + 'gwfish_medians/' + event + '/' + label_cov)[0, :, :]
    samples = np.random.multivariate_normal(mns, cov, samples_len[event])
    samples_df = pd.DataFrame(samples, columns = params)
    samples_df.to_hdf(PATH_TO_RESULTS + 'posterior_samples/fisher_samples/fisher_samples_%s_%s.hdf5' %(estimator, event), mode='w', key='root')

### Fisher + Priors samples

In [25]:
with open(PATH_TO_INFO + 'chirp_mass_priors_%s.pkl' %waveform, 'rb') as f:
    chirp_mass_priors = pickle.load(f)

In [26]:
with open(PATH_TO_INFO + 'geocent_time_priors_%s.pkl' %waveform, 'rb') as f:
    geocent_time_priors = pickle.load(f)

In [27]:
# Build priors range dictionary
my_priors = {}
for event in events:
    event_dict = {
        'chirp_mass': np.array([float(chirp_mass_priors[event][0]), float(chirp_mass_priors[event][1])]),
        'mass_ratio': np.array([0.05, 1.0]),
        'luminosity_distance': np.array([10, 10000]),
        'dec': np.array([-np.pi/2, np.pi/2]),
        'ra': np.array([0, 2*np.pi]),
        'theta_jn': np.array([0, np.pi]),
        'psi': np.array([0, np.pi]),
        'phase': np.array([0, 2*np.pi]),
        'geocent_time': np.array([float(geocent_time_priors[event][0]), float(geocent_time_priors[event][1])]),
        'a_1': np.array([0, 0.99]),
        'a_2': np.array([0, 0.99]),
        'tilt_1': np.array([0, np.pi]),
        'tilt_2': np.array([0, np.pi]),
        'phi_12': np.array([0, 2*np.pi]),
        'phi_jl': np.array([0, 2*np.pi])
    }
    my_priors[event] = event_dict

In [28]:
min_array = np.array([-np.inf, 0, 0, -np.pi/2, 0, 0, 0, 0, -np.inf, 0, 0, 0, 0, 0, 0])
max_array = np.array([np.inf, 1, 20000, np.pi/2, 2*np.pi, np.pi, np.pi, 2*np.pi, np.inf, 1, 1, np.pi, np.pi, 2*np.pi, 2*np.pi])

for event in events:
    label_err = get_label(detectors, event, estimator, 0, 'errors')
    label_cov = get_label(detectors, event, estimator, 0, 'inv_fishers')
    data = pd.read_csv(PATH_TO_RESULTS + 'gwfish_medians/' + event + '/' + label_err, names = lbs_errs, delimiter=' ', skiprows=1)
    mns = data[params].iloc[0].to_numpy()
    cov = np.load(PATH_TO_RESULTS + 'gwfish_medians/' + event +'/' + label_cov)[0, :, :]
    samples_tmvn = get_samples_from_TMVN(min_array, max_array, mns, cov, samples_len[event])
    new_df = pd.DataFrame(samples_tmvn.T, columns = params)
    posteriors = get_posteriors(new_df, my_priors[event], samples_len[event])
    posteriors_df = pd.DataFrame(posteriors, columns = params)
    posteriors_df.to_hdf(PATH_TO_RESULTS + 'posterior_samples/fisher_plus_priors_samples/fisher_plus_priors_samples_%s_%s.hdf5' %(estimator, event), mode='w', key='root')